# Imports and Dataset Preparation

**In this section, we start by importing the necessary libraries. We define the dataset directory and create separate directories for training and validation datasets. We split the dataset into training and validation sets using a ratio of 80:20. The images are then moved to their respective directories.**

In [1]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score


import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.models as models

# we define dataset directory
data_dir = '/kaggle/input/indian-actor-images-dataset/Bollywood Actor Images/Bollywood Actor Images'

# defining paths for train and validation sets
base_dir = '/kaggle/working'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

# create directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# in this part we get list of actor directories
actor_dirs = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]

# split ratio 80% for train
train_ratio = 0.8

for actor in actor_dirs:
    actor_path = os.path.join(data_dir, actor)
    images = os.listdir(actor_path)
    
    train_images, val_images = train_test_split(images, train_size=train_ratio, random_state=42)

    # Create actor directories in train and val directories
    train_actor_dir = os.path.join(train_dir, actor)
    val_actor_dir = os.path.join(val_dir, actor)
    os.makedirs(train_actor_dir, exist_ok=True)
    os.makedirs(val_actor_dir, exist_ok=True)

    # move training images
    for img in train_images:
        src_path = os.path.join(actor_path, img)
        dst_path = os.path.join(train_actor_dir, img)
        shutil.copyfile(src_path, dst_path)

    # move validation images
    for img in val_images:
        src_path = os.path.join(actor_path, img)
        dst_path = os.path.join(val_actor_dir, img)
        shutil.copyfile(src_path, dst_path)

# Data Transformations and DataLoader Setup

**Here, we define the data transformations to be applied to the images. The images are resized, converted to tensors, and normalized. We then load the datasets using ImageFolder and create data loaders for both training and validation datasets.**

In [2]:
# Define transforms and load data
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet standard input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


# Model Setup and Loading Pretrained Weights ResNet18

**Here, we load the ResNet-18 model and modify it to suit our specific task. The final fully connected layer of the model is adjusted to match the number of classes (135). We load pretrained weights from a specified file path and modify the state dictionary to align with our model's final layer. The model is then moved to the appropriate device (GPU if available, otherwise CPU).**

In [3]:
# load ResNet and modifing it
resnet18 = models.resnet18(weights=None)  # initialize without pre-trained weights first
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, 135)  # 135 classes

# load pretrained weights 
pretrained_weights_path = '/kaggle/input/resnet18/resnet18-f37072fd.pth'
assert os.path.isfile(pretrained_weights_path), f"Error: Pretrained weights file '{pretrained_weights_path}' not found."
state_dict = torch.load(pretrained_weights_path, map_location=torch.device('cpu'))

# adjust the state_dict for the final fully connected layer
state_dict['fc.weight'] = state_dict['fc.weight'][:135, :]
state_dict['fc.bias'] = state_dict['fc.bias'][:135]

resnet18.load_state_dict(state_dict, strict=False)  

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet18 = resnet18.to(device)

# Training Loop with Validation Accuracy Checks ResNet18

**This section defines the training loop and includes validation accuracy checks. We set up the loss function (Cross-Entropy Loss) and the optimizer (Adam). The model is trained for a specified number of epochs, and the training loss is calculated and printed for each epoch. After each epoch, the model is evaluated on the validation set to check the accuracy.**

In [4]:
# Defining loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet18.parameters(), lr=0.001)

In [5]:
# training loops
num_epochs = 25
for epoch in range(num_epochs):
    resnet18.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = resnet18(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Validation loop
resnet18.eval()
all_labels = []
all_predictions = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = resnet18(inputs)
        _, predicted = torch.max(outputs.data, 1)
        
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

accuracy = accuracy_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f'Validation Accuracy: {accuracy * 100:.2f}%')
print(f'F1 Score: {f1:.4f}')


Epoch [1/25], Loss: 4.8242
Epoch [2/25], Loss: 3.7764
Epoch [3/25], Loss: 2.9008
Epoch [4/25], Loss: 1.9356
Epoch [5/25], Loss: 1.0292
Epoch [6/25], Loss: 0.3666
Epoch [7/25], Loss: 0.1061
Epoch [8/25], Loss: 0.0160
Epoch [9/25], Loss: 0.0039
Epoch [10/25], Loss: 0.0020
Epoch [11/25], Loss: 0.0015
Epoch [12/25], Loss: 0.0012
Epoch [13/25], Loss: 0.0009
Epoch [14/25], Loss: 0.0008
Epoch [15/25], Loss: 0.0007
Epoch [16/25], Loss: 0.0006
Epoch [17/25], Loss: 0.0005
Epoch [18/25], Loss: 0.0005
Epoch [19/25], Loss: 0.0004
Epoch [20/25], Loss: 0.0004
Epoch [21/25], Loss: 0.0003
Epoch [22/25], Loss: 0.0003
Epoch [23/25], Loss: 0.0003
Epoch [24/25], Loss: 0.0003
Epoch [25/25], Loss: 0.0002
Validation Accuracy: 52.59%
F1 Score: 0.5207


# Saving the Model

**Finally, the model will be saved.**

In [6]:
torch.save(resnet18.state_dict(), 'indian_actor_resnet18.pth')

# Model Setup and Loading Pretrained Weights MobileNetV2 

**in this part we train our model with MobileNetV2 instead of ResNet18**

In [7]:
mobilenet_v2 = models.mobilenet_v2(weights=None)  # Initialize without pre-trained weights first
num_ftrs = mobilenet_v2.classifier[1].in_features
mobilenet_v2.classifier[1] = nn.Linear(num_ftrs, 135)  # 135 classes
mobilenet_v2 = mobilenet_v2.to(device)

# loading pretrained weights 
pretrained_weights_path = '/kaggle/input/mobilenet-v2-b0353104-pth/mobilenet_v2-b0353104.pth'
if os.path.isfile(pretrained_weights_path):
    state_dict = torch.load(pretrained_weights_path, map_location=torch.device('cpu'))
    state_dict['classifier.1.weight'] = state_dict['classifier.1.weight'][:135, :]
    state_dict['classifier.1.bias'] = state_dict['classifier.1.bias'][:135]
    mobilenet_v2.load_state_dict(state_dict, strict=False)

# Training Loop with Validation Accuracy Checks MobileNetV2


**Like before we define traning loops here**

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mobilenet_v2.parameters(), lr=0.001)

In [9]:
for epoch in range(num_epochs):
    mobilenet_v2.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = mobilenet_v2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

Epoch [1/25], Loss: 4.8239
Epoch [2/25], Loss: 3.5963
Epoch [3/25], Loss: 2.8629
Epoch [4/25], Loss: 2.2443
Epoch [5/25], Loss: 1.7180
Epoch [6/25], Loss: 1.3453
Epoch [7/25], Loss: 1.0369
Epoch [8/25], Loss: 0.7962
Epoch [9/25], Loss: 0.5841
Epoch [10/25], Loss: 0.4300
Epoch [11/25], Loss: 0.3885
Epoch [12/25], Loss: 0.3876
Epoch [13/25], Loss: 0.3592
Epoch [14/25], Loss: 0.2989
Epoch [15/25], Loss: 0.2691
Epoch [16/25], Loss: 0.2952
Epoch [17/25], Loss: 0.3082
Epoch [18/25], Loss: 0.2564
Epoch [19/25], Loss: 0.1977
Epoch [20/25], Loss: 0.1859
Epoch [21/25], Loss: 0.2392
Epoch [22/25], Loss: 0.3037
Epoch [23/25], Loss: 0.2010
Epoch [24/25], Loss: 0.1654
Epoch [25/25], Loss: 0.2273


**validations are here**

In [10]:
# validation loop
mobilenet_v2.eval()
all_labels = []
all_predictions = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = mobilenet_v2(inputs)
        _, predicted = torch.max(outputs.data, 1)
        
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

accuracy = accuracy_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f'MobileNetV2 Validation Accuracy: {accuracy * 100:.2f}%')
print(f'MobileNetV2 F1 Score: {f1:.4f}')


MobileNetV2 Validation Accuracy: 53.48%
MobileNetV2 F1 Score: 0.5318


# Saving the Model

**finally we save our model**

In [11]:
torch.save(mobilenet_v2.state_dict(), 'indian_actor_mobilenetv2.pth')